# Behavior Test (Insertion, Deletion, Replacement) Analysis

In [1]:
import pandas as pd

In [2]:
filenames = ['_'.join(["spanish",i,j]) + ".csv" for i in ["insertion", "deletion", "replacement"] for j in ["saint", "dkt1"]]

In [3]:
filenames # files in notion

['spanish_insertion_saint.csv',
 'spanish_insertion_dkt1.csv',
 'spanish_deletion_saint.csv',
 'spanish_deletion_dkt1.csv',
 'spanish_replacement_saint.csv',
 'spanish_replacement_dkt1.csv']

In [4]:
result_list = []
for fn in filenames:
    result_list.append(pd.read_csv(fn))

In [5]:
x = result_list[0]

In [6]:
x.columns

Index(['Unnamed: 0', 'user_id', 'item_id', 'timestamp', 'correct', 'skill_id',
       'orig_user_id', 'orig_idx', 'is_perturbed', 'model_pred', 'testpass',
       'orig_prob', 'all'],
      dtype='object')

In [10]:
threshold = 0

for df in result_list:
    corr_df = df.loc[df['is_perturbed'] == 1]
    incorr_df = df.loc[df['is_perturbed'] == -1]
    corr_testpass = (corr_df['model_pred'] >= corr_df['orig_prob'] - threshold).values.tolist()
    incorr_testpass = (incorr_df['model_pred'] <= incorr_df['orig_prob'] + threshold).values.tolist()
    print(corr_testpass.count(True), '|', corr_testpass.count(False), '->', corr_testpass.count(True)/len(corr_testpass))
    print(incorr_testpass.count(True), '|', incorr_testpass.count(False), '->', incorr_testpass.count(True)/len(incorr_testpass))
    print((corr_testpass.count(True)+incorr_testpass.count(True)) / len(df))
    print()

25939 | 25849 -> 0.5008689271645941
26506 | 25282 -> 0.51181740943848
0.506343168301537

27037 | 24751 -> 0.5220707499806905
23588 | 28200 -> 0.45547231018768825
0.4887715300841894

5287 | 3707 -> 0.5878363353346675
49863 | 43160 -> 0.5360287240789913
0.5405961751472794

3653 | 5341 -> 0.4061596619968868
45661 | 47362 -> 0.4908570998570246
0.4833900232314222

5323 | 3671 -> 0.591839003780298
49883 | 43140 -> 0.5362437246702428
0.5411451032671025

3687 | 5307 -> 0.40993995997331556
45746 | 47277 -> 0.491770852369844
0.48455649548604646



## Model Summary

DKT original test auc : 0.8278944242188684

SAINT original test auc : 0.847428944098048

test pass probability (threshold=0.0)

| model | test AUC | insertion | deletion | replacement |
| DKT   | 0.8279   | 48.88%    | 48.34%   | 48.45%      |
| SAINT | 0.8474   | 50.63%    | 54.06%   | 54.11%      |